Import libraries


In [1]:
# Analysis modules
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

### Load data


In [2]:
path_to_data  = '../../data/OriginalData.csv'
data = pd.read_csv(path_to_data)

print(data.info())

print('\nKEYS\n----')
for i in data.keys():
    print(i)
print()

display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15748 entries, 0 to 15747
Data columns (total 49 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Dates                             15748 non-null  object 
 1   symbol                            15748 non-null  object 
 2   PX_LAST                           15748 non-null  float64
 3   Dividend_Per_Share                15748 non-null  float64
 4   TOTAL_EQUITY                      15748 non-null  float64
 5   CUR_MKT_CAP                       15748 non-null  float64
 6   gics_sector_name                  15748 non-null  object 
 7   gics_sub_industry_name            15748 non-null  object 
 8   gics_industry_name                15748 non-null  object 
 9   gics_industry_group_name          15748 non-null  object 
 10  gics_industry                     15748 non-null  int64  
 11  gics_sector                       15748 non-null  int64  
 12  Year

,Dates,symbol,PX_LAST,Dividend_Per_Share,TOTAL_EQUITY,CUR_MKT_CAP,gics_sector_name,gics_sub_industry_name,gics_industry_name,gics_industry_group_name,...,NUM_CEOS_EQUIV_INC_COMP_PD,CEO_TENURE_AS_OF_FY_END,TOT_COMP_AW_TO_CFO_EQUIV,TOT_COMP_AW_TO_COO_EQUIV,TOTAL_BOD_COMPENSATION_AWARDED,TOTAL_BOD_FEES_PAID_IN_CASH,TOTAL_BOD_STOCK_AWARDS_GIVEN,NUMBER_OF_BOD_CHANGES_DURING_FY,DATE_OF_LAST_BOD_CHANGE,NUM_DIR_INC_IN_CMPNSTN_PD
0,2014-04-11,AAPL,18.5580,0.1089,120179.000,463486.4114,Information Technology,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals",Technology Hardware & Equipment,...,1,3.7500,18520521.0,0,2638340.0,697500.0,1906645.0,2,2014-07-17,8
1,2014-04-11,AMT,80.5500,0.3200,3701.605,31870.1921,Real Estate,Telecom Tower REITs,Specialized REITs,Equity Real Estate Investment Trusts (REITs),...,1,11.2500,4904866.0,0,1975088.0,775000.0,600071.0,1,2014-12-18,9
2,2014-04-11,AMZN,15.5870,0.0000,10328.000,143440.4076,Consumer Discretionary,Broadline Retail,Broadline Retail,Consumer Discretionary Distribution & Retail,...,1,18.6670,5962307.0,0,797000.0,0.0,797000.0,1,2014-07-31,9
3,2014-04-11,BHP,58.8727,0.0000,82279.000,180544.3056,Materials,Diversified Metals & Mining,Metals & Mining,Materials,...,1,1.1667,4639000.0,0,4695000.0,3617000.0,0.0,1,2014-04-15,13
4,2014-04-11,CAT,101.4500,0.6000,20369.000,64707.0750,Industrials,Construction Machinery & Heavy Transportation ...,Machinery,Capital Goods,...,1,4.5000,5286968.0,0,3206983.0,1714174.0,1375110.0,1,2014-12-31,11


### Produce subset dataframe

In [3]:
# Extract these keys from the original data set to produce a smaller set containing only what we need
relevant_data_keys = ['Dates', 
                 'Year', 
                 'symbol',
                 'PX_LAST', 
                 'gics_sector_name', 
                 'DATE_OF_LAST_EXECUTIVE_CHANGE',
                 'TOT_STK_AWD_GIVEN_TO_CEO_EQUIV',
                 'TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV',
                 'TOT_BONUSES_PAID_TO_CEO_EQUIV',
                 'TOT_N_EQT_INCENT_GVN_TO_CEO_EQ']

# Produce a subset dataframe containing only the necessary data
subset_data = data[relevant_data_keys].copy()

# Cast both dates columns to datetime64[ns]
# 'Dates', 'DATE_OF_LAST_EXECUTIVE_CHANGE'
subset_data['Dates'] = pd.to_datetime(subset_data['Dates'])
subset_data['DATE_OF_LAST_EXECUTIVE_CHANGE'] = pd.to_datetime(subset_data['DATE_OF_LAST_EXECUTIVE_CHANGE'])

subset_data.info()
subset_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15748 entries, 0 to 15747
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Dates                           15748 non-null  datetime64[ns]
 1   Year                            15748 non-null  int64         
 2   symbol                          15748 non-null  object        
 3   PX_LAST                         15748 non-null  float64       
 4   gics_sector_name                15748 non-null  object        
 5   DATE_OF_LAST_EXECUTIVE_CHANGE   15748 non-null  datetime64[ns]
 6   TOT_STK_AWD_GIVEN_TO_CEO_EQUIV  15748 non-null  float64       
 7   TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV  15748 non-null  int64         
 8   TOT_BONUSES_PAID_TO_CEO_EQUIV   15748 non-null  int64         
 9   TOT_N_EQT_INCENT_GVN_TO_CEO_EQ  15748 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(4), object(2)
memory usage: 1.

,Dates,Year,symbol,PX_LAST,gics_sector_name,DATE_OF_LAST_EXECUTIVE_CHANGE,TOT_STK_AWD_GIVEN_TO_CEO_EQUIV,TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV,TOT_BONUSES_PAID_TO_CEO_EQUIV,TOT_N_EQT_INCENT_GVN_TO_CEO_EQ
0,2014-04-11,2014,AAPL,18.5580,Information Technology,2014-05-29,0.0,0,0,6700000
1,2014-04-11,2014,AMT,80.5500,Real Estate,2013-12-31,4500051.0,4500007,0,2574000
2,2014-04-11,2014,AMZN,15.5870,Consumer Discretionary,2006-04-30,0.0,0,0,0
3,2014-04-11,2014,BHP,58.8727,Materials,2014-03-03,3338000.0,0,1568000,0
4,2014-04-11,2014,CAT,101.4500,Industrials,2013-01-01,0.0,8377481,0,4913288
...,...,...,...,...,...,...,...,...,...,...
15743,2023-12-29,2023,UNH,526.4700,Health Care,2022-03-01,15000970.0,5000114,0,1800000
15744,2023-12-29,2023,V,260.3500,Financials,2023-08-01,28004734.0,10707531,0,10312500
15745,2023-12-29,2023,WFC,49.2200,Financials,2020-10-01,16634168.0,0,0,6625000
15746,2023-12-29,2023,WMT,52.5500,Consumer Staples,2022-06-06,30103720.0,0,0,4627582


### Isolate data for processing

In [4]:
# Create mini dataframe viewing just one company
aapl_df = subset_data[subset_data['symbol'] == 'AAPL'].copy()

display(aapl_df.info())
display(aapl_df)

<class 'pandas.core.frame.DataFrame'>
Index: 508 entries, 0 to 15717
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Dates                           508 non-null    datetime64[ns]
 1   Year                            508 non-null    int64         
 2   symbol                          508 non-null    object        
 3   PX_LAST                         508 non-null    float64       
 4   gics_sector_name                508 non-null    object        
 5   DATE_OF_LAST_EXECUTIVE_CHANGE   508 non-null    datetime64[ns]
 6   TOT_STK_AWD_GIVEN_TO_CEO_EQUIV  508 non-null    float64       
 7   TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV  508 non-null    int64         
 8   TOT_BONUSES_PAID_TO_CEO_EQUIV   508 non-null    int64         
 9   TOT_N_EQT_INCENT_GVN_TO_CEO_EQ  508 non-null    int64         
dtypes: datetime64[ns](2), float64(2), int64(4), object(2)
memory usage: 43.7+ KB


None

,Dates,Year,symbol,PX_LAST,gics_sector_name,DATE_OF_LAST_EXECUTIVE_CHANGE,TOT_STK_AWD_GIVEN_TO_CEO_EQUIV,TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV,TOT_BONUSES_PAID_TO_CEO_EQUIV,TOT_N_EQT_INCENT_GVN_TO_CEO_EQ
0,2014-04-11,2014,AAPL,18.558,Information Technology,2014-05-29,0.0,0,0,6700000
31,2014-04-18,2014,AAPL,18.748,Information Technology,2014-05-29,0.0,0,0,6700000
62,2014-04-25,2014,AAPL,20.426,Information Technology,2014-05-29,0.0,0,0,6700000
93,2014-05-02,2014,AAPL,21.164,Information Technology,2014-05-29,0.0,0,0,6700000
124,2014-05-09,2014,AAPL,20.912,Information Technology,2014-05-29,0.0,0,0,6700000
...,...,...,...,...,...,...,...,...,...,...
15593,2023-12-01,2023,AAPL,191.240,Information Technology,2019-02-05,46970283.0,0,0,10713450
15624,2023-12-08,2023,AAPL,195.710,Information Technology,2019-02-05,46970283.0,0,0,10713450
15655,2023-12-15,2023,AAPL,197.570,Information Technology,2019-02-05,46970283.0,0,0,10713450
15686,2023-12-22,2023,AAPL,193.600,Information Technology,2019-02-05,46970283.0,0,0,10713450


### Compute returns
Cash incentive for CEO

Compensation ratio is computed as 
$\frac{Cash}{Cash + Equity}$

where 1.0 denotes 100% cash compensation, and 0.0 denotes 100% equity compensation

In [5]:
# add columns to aapl_df
aapl_df.loc[:,"equity_Compensation_CEO"] = aapl_df['TOT_STK_AWD_GIVEN_TO_CEO_EQUIV']+aapl_df["TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV"]
aapl_df.loc[:,"cash_incentive_CEO"] = aapl_df['TOT_BONUSES_PAID_TO_CEO_EQUIV']+aapl_df['TOT_N_EQT_INCENT_GVN_TO_CEO_EQ']
aapl_df.loc[:,"Variable_Pay_CEO"] = aapl_df["cash_incentive_CEO"]+aapl_df["equity_Compensation_CEO"]+0.01
aapl_df.loc[:,"cash_incentive_CEO_percent"] = aapl_df["cash_incentive_CEO"]/aapl_df["Variable_Pay_CEO"]

display(aapl_df)

,Dates,Year,symbol,PX_LAST,gics_sector_name,DATE_OF_LAST_EXECUTIVE_CHANGE,TOT_STK_AWD_GIVEN_TO_CEO_EQUIV,TOT_OPT_AWD_GIVEN_TO_CEO_EQUIV,TOT_BONUSES_PAID_TO_CEO_EQUIV,TOT_N_EQT_INCENT_GVN_TO_CEO_EQ,equity_Compensation_CEO,cash_incentive_CEO,Variable_Pay_CEO,cash_incentive_CEO_percent
0,2014-04-11,2014,AAPL,18.558,Information Technology,2014-05-29,0.0,0,0,6700000,0.0,6700000,6700000.01,1.000000
31,2014-04-18,2014,AAPL,18.748,Information Technology,2014-05-29,0.0,0,0,6700000,0.0,6700000,6700000.01,1.000000
62,2014-04-25,2014,AAPL,20.426,Information Technology,2014-05-29,0.0,0,0,6700000,0.0,6700000,6700000.01,1.000000
93,2014-05-02,2014,AAPL,21.164,Information Technology,2014-05-29,0.0,0,0,6700000,0.0,6700000,6700000.01,1.000000
124,2014-05-09,2014,AAPL,20.912,Information Technology,2014-05-29,0.0,0,0,6700000,0.0,6700000,6700000.01,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15593,2023-12-01,2023,AAPL,191.240,Information Technology,2019-02-05,46970283.0,0,0,10713450,46970283.0,10713450,57683733.01,0.185727
15624,2023-12-08,2023,AAPL,195.710,Information Technology,2019-02-05,46970283.0,0,0,10713450,46970283.0,10713450,57683733.01,0.185727
15655,2023-12-15,2023,AAPL,197.570,Information Technology,2019-02-05,46970283.0,0,0,10713450,46970283.0,10713450,57683733.01,0.185727
15686,2023-12-22,2023,AAPL,193.600,Information Technology,2019-02-05,46970283.0,0,0,10713450,46970283.0,10713450,57683733.01,0.185727


### Plot compensation ratio to returns

In [ ]:
# Get Numpy array of weekly returns
weekly_returns = aapl_df['PX_LAST'].pct_change() * 100
weekly_returns = weekly_returns.dropna()
weekly_returns = weekly_returns.to_numpy()

# Get Numpy array of CEO cash incentive percentages
cash_incentive = aapl_df['cash_incentive_CEO_percent'].iloc[1:]

[ 1.02381722e+00  8.95028803e+00  3.61304220e+00 -1.19070119e+00
  2.04667177e+00  2.77881912e+00  3.07299503e+00  1.98611050e+00
 -1.02359473e+00 -4.03155127e-01  1.17476241e+00  2.23091977e+00
  1.26339969e+00 -8.27557236e-01  3.43104033e+00 -1.57670571e+00
 -1.44800899e+00  3.41988600e+00  3.40885895e+00  1.16462692e+00
 -3.44195122e+00  2.71591965e+00 -6.88569742e-01 -2.06022187e-01
 -1.12355090e+00  1.11624172e+00 -3.03776357e+00  7.72790564e+00
  2.64208325e+00  9.37037037e-01  4.74076248e+00  2.00735681e+00
  2.11209561e+00 -3.30609088e+00 -4.58086957e+00  1.86636533e+00
  1.97888710e+00 -4.08800618e+00  2.45124941e+00 -5.37442417e+00
  6.59295041e+00  3.69976987e+00  1.51246159e+00  6.85097367e+00
  1.90116462e+00 -8.00024711e-01 -1.44792153e+00 -2.37598736e+00
  1.86743479e+00 -2.10325655e+00  1.67786324e+00  1.42036387e+00
 -1.84736428e+00  4.43119148e+00 -1.02241326e+00 -1.02987251e+00
  9.02679831e-01  2.92610195e+00 -1.70514562e+00 -1.24961621e+00
 -1.15039020e+00 -4.49784